In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import joblib
import warnings 
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, r2_score, max_error
from sklearn.ensemble import RandomForestClassifier



# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [ ]:
df.describe().T

In [ ]:
sns.heatmap(df.corr(), cmap='RdYlGn')

In [ ]:
selected_features = df[["koi_period","koi_period_err1","koi_period_err2","koi_time0bk","koi_time0bk_err1",
                     "koi_steff_err2","koi_slogg","koi_slogg_err1","koi_slogg_err2","koi_srad",
                     "koi_srad_err1","koi_srad_err2","ra","dec","koi_kepmag"]]

In [ ]:
selected_features.corr()

In [ ]:
sns.heatmap(selected_features.corr(), cmap='RdYlGn')

In [ ]:
# Check for null values
df.info(verbose=True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(df['koi_disposition'])

In [ ]:
# Set features. This will also be used as your x values.


# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
# create features
X = df.drop("koi_disposition", axis=1)
feature_names = X.columns
# create labels
y = df["koi_disposition"]

print("Shape: ", X.shape, y.shape)


Shape:  (6991, 40) (6991,)


In [4]:
# split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [5]:
# scale the data
X_scaler = MinMaxScaler().fit(X_train)


In [6]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Train the Model



In [7]:
# create a Gaussian Classifier with default n_esimators value
model = RandomForestClassifier(n_estimators = 100)

# train the model
model = model.fit(X_train_scaled, y_train)

print(f"Training Data Score: {model.score(X_train_scaled, y_train):.4f}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test):.4f}")


Training Data Score: 1.0000
Testing Data Score: 0.8970


In [8]:
# predict test data set
y_pred = model.predict(X_test_scaled)


In [ ]:
# check performance of model with classification report
print(classification_report(y_test, y_pred))


In [ ]:
confusion_matrix(y_test,y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
importances = model.feature_importances_
feature_imp = pd.Series(model.feature_importances_, index = feature_names).sort_values(ascending=False)


In [ ]:
# Creating a bar plot
plt.figure(figsize=(10,12))
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [24]:
# example of grid searching key hyperparametres for KNeighborsClassifier
max_features = []

# set range 1 to half the number of input features ie 20
for i in range(1,20):
    max_features.append(i)

# define grid search
param_grid = {'max_features': max_features, 
              'n_estimators': [100, 200],
              'max_depth': [10, 20]
             }


In [25]:
grid = GridSearchCV(model, param_grid, n_jobs = -1, verbose = 3)


In [26]:
grid.get_params(deep=True)

{'cv': None,
 'error_score': nan,
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(),
 'iid': 'deprecated',
 'n_jobs': -1,
 'param_grid': {'max_features': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19],
  'n_estimators': [100, 200],
  'max_depth': [10, 20]},
 'pre_dispatch': '2*n_jobs',
 

In [27]:
# Train the model with GridSearch (cross validate to determine hyper parameter values for best accuracy)
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 76 candidates, totalling 380 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 380 out of 380 | elapsed:  9.5min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 20],
                         'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100, 200]},
             verbose=3)

In [28]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

{'max_depth': 20, 'max_features': 16, 'n_estimators': 200}
0.9061636309389531
RandomForestClassifier(max_depth=20, max_features=16, n_estimators=200)


In [29]:
print(f"Training Data Score: {grid.score(X_train_scaled, y_train):.4f}")
print(f"Testing Data Score: {grid.score(X_test_scaled, y_test):.4f}")

Training Data Score: 0.9939
Testing Data Score: 0.8959


In [31]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [32]:
# check performance of model with classification report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.76      0.79       422
     CONFIRMED       0.79      0.83      0.81       450
FALSE POSITIVE       0.98      0.99      0.99       876

      accuracy                           0.90      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.90      0.90      0.89      1748



In [33]:
confusion_matrix(y_test, predictions)
pd.crosstab(y_test, predictions, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,CANDIDATE,CONFIRMED,FALSE POSITIVE,All
True,,,,
CANDIDATE,320,97,5,422
CONFIRMED,61,375,14,450
FALSE POSITIVE,3,2,871,876
All,384,474,890,1748


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

filename = 'your_name.sav'
joblib.dump(your_model, filename)